In [ ]:
from sklearn import preprocessing
import pandas as pd
import numpy as np
from math import pi, sin, cos
from datetime import datetime

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping


In [ ]:
# Data can be downloaded at https://www.kaggle.com/c/bike-sharing-demand/data

In [ ]:
train = pd.read_csv('../Data/bike-sharing/train.csv')
test = pd.read_csv('../Data/bike-sharing/test.csv')
data = pd.concat([train, test])
test_split = train.shape[0]

In [ ]:
data.head()

In [ ]:
data['hour'] = data.apply(lambda x: datetime.strptime(x['datetime'], '%Y-%m-%d %H:%M:%S').hour, axis=1)
data['weekday'] = data.apply(lambda x: datetime.strptime(x['datetime'], '%Y-%m-%d %H:%M:%S').weekday(), axis=1)
data['month'] = data.apply(lambda x: datetime.strptime(x['datetime'], '%Y-%m-%d %H:%M:%S').month, axis=1)
data['year'] = data.apply(lambda x: datetime.strptime(x['datetime'], '%Y-%m-%d %H:%M:%S').year, axis=1)

In [ ]:
data['hour_sin'] = data.apply(lambda x: sin(x['hour'] / 24.0 * 2 * pi), axis=1)
data['hour_cos'] = data.apply(lambda x: cos(x['hour'] / 24.0 * 2 * pi), axis=1)
data['weekday_sin'] = data.apply(lambda x: sin(x['weekday'] / 7.0 * 2 * pi), axis=1)
data['weekday_cos'] = data.apply(lambda x: cos(x['weekday'] / 7.0 * 2 * pi), axis=1)
data['month_sin'] = data.apply(lambda x: sin(((x['month'] - 5) % 12) / 12.0 * 2 * pi), axis=1)
data['month_cos'] = data.apply(lambda x: cos(((x['month'] - 5) % 12) / 12.0 * 2 * pi), axis=1)
data['season_sin'] = data.apply(lambda x: sin(((x['season'] - 3) % 4) / 4.0 * 2 * pi), axis=1)
data['season_cos'] = data.apply(lambda x: cos(((x['season'] - 3) % 4) / 4.0 * 2 * pi), axis=1)

In [ ]:
X_train = data[:test_split].drop(['datetime', 'casual', 'registered', 'count'], inplace=False, axis=1)
X_test = data[test_split:].drop(['datetime', 'casual', 'registered', 'count'], inplace=False, axis=1)
y_train = data['count'][:test_split]
y_test = data['count'][test_split:]

In [ ]:
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = Sequential()
model.add(Dense(200, input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(1))

opt = Adam()
model.compile(loss='mean_squared_logarithmic_error', optimizer=opt)

In [ ]:
n_epochs = 1000
batch_size = 128

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5)]
history = model.fit(X_train, y_train, shuffle=True, epochs=n_epochs, batch_size=batch_size, validation_split=0.1, verbose=1)